In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
plt.style.use('ggplot')
from tensorflow import keras

# Load the dataset
data = pd.read_csv('Twitter_Data.csv', encoding='utf-8')
data = data.dropna()

# Preprocess the dataset
labels = pd.get_dummies(data.category)
labels.columns = ["negative", "neutral", "positive"]
data = data.drop(columns="category")

# Tokenization and padding
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=8150, lower=True, split=" ", oov_token="~")
tokenizer.fit_on_texts(data["clean_text"])
data["clean_text"] = tokenizer.texts_to_sequences(data["clean_text"])
tweets = pad_sequences(data["clean_text"])

# Split the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tweets, labels, test_size=0.15)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2)

# Build the model
model = keras.models.Sequential([
    keras.layers.Embedding(input_dim=8150, output_dim=32),
    keras.layers.GRU(128),
    keras.layers.Dense(128, activation="relu", kernel_initializer="he_normal", kernel_regularizer=tf.keras.regularizers.l1(0.01)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(3, activation="softmax", kernel_initializer="glorot_normal")
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=7, validation_data=(X_valid, y_valid),
    callbacks=[keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)]
)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

Epoch 1/7
3464/3464 [==============================] - 60s 17ms/step - loss: 0.9647 - accuracy: 0.8413 - val_loss: 0.3747 - val_accuracy: 0.9223
Epoch 2/7
3464/3464 [==============================] - 29s 8ms/step - loss: 0.3556 - accuracy: 0.9433 - val_loss: 0.2979 - val_accuracy: 0.9547
Epoch 3/7
3464/3464 [==============================] - 27s 8ms/step - loss: 0.3023 - accuracy: 0.9602 - val_loss: 0.2820 - val_accuracy: 0.9624
Epoch 4/7
3464/3464 [==============================] - 29s 8ms/step - loss: 0.2769 - accuracy: 0.9662 - val_loss: 0.2826 - val_accuracy: 0.9629
Epoch 5/7
3464/3464 [==============================] - 27s 8ms/step - loss: 0.2617 - accuracy: 0.9701 - val_loss: 0.2782 - val_accuracy: 0.9652
Epoch 6/7
3464/3464 [==============================] - 27s 8ms/step - loss: 0.2477 - accuracy: 0.9738 - val_loss: 0.2933 - val_accuracy: 0.9650
Epoch 7/7
764/764 [==============================] - 3s 4ms/step - loss: 0.2973 - accuracy: 0.9629
Test loss: 0.2973049283027649
Test a

In [36]:
# User input for prediction
user_input = input("Enter a text: ")
user_input_tokens = tokenizer.texts_to_sequences([user_input])
user_input_padded = pad_sequences(user_input_tokens, maxlen=tweets.shape[1], padding='post')

# Make predictions
predictions = model.predict(user_input_padded)

# Post-process predictions
predicted_class = np.argmax(predictions, axis=-1)

# Define a mapping of class indices to sentiment labels
class_to_sentiment = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}

# Get the corresponding sentiment label
predicted_sentiment = class_to_sentiment[predicted_class[0]]

# Print the predicted sentiment
print("Predicted sentiment:", predicted_sentiment)


Enter a text: Five years ago, Union Berlin were getting ready to play Arminia Bielefeld in a 2. Bundesliga match.  Today, they're set to face Real Madrid at the Bernabéu in what will be the first Champions League match in the club's history.
1/1 [==============================] - 0s 36ms/step
Predicted sentiment: Positive
